# Tableau project data cleaning + preparation

#### Around 56 million people die each year. What caused their death?

In [ ]:
# Imports

import pandas as pd
import re

In [ ]:
# Pandas options

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Clean death causes dataset

In [ ]:
causes = pd.read_csv('dataset_deaths_by_cause.csv')

causes.head()

In [ ]:
# Clean column names

causes.columns = [re.sub('^\w{6} - | - S.+', '', col) for col in causes.columns]

causes = causes.rename(columns = {'Entity': 'Country', 'Number of executions (Amnesty International)': 'Executions'})
causes = causes.rename(columns = {"Alzheimer's disease and other dementias": 'Dementias', 'Cirrhosis and other chronic liver diseases': 'Chronic liver diseases'})

causes.head()

In [ ]:
# Clean country-list

# print(set(causes['Country']))

In [ ]:
causes_country_remove = ['Sub-Saharan Africa', 'Eastern Europe', 'South Asia (WB)', 'African Region (WHO)', 'West Germany (FRG)', 'World Bank Upper Middle Income', 'Central America & Caribbean', 'Czechoslovakia', 'World Bank Low Income', 'Australasia & Oceania', 'Western Pacific Region (WHO)', 'Yugoslavia', 'European Region (WHO)', 'World (excluding China)', 'England', 'Northern Ireland', 'Scotland', 'Wales', 'World Bank High Income', 'East Asia', 'USSR', 'East Germany (GDR)', 'North America (WB)', 'Middle East & North Africa', 'Southeast Asia', 'Western Sahara', 'World Bank Lower Middle Income', 'Region of the Americas (WHO)', 'G20', 'International', 'South-East Asia Region (WHO)', 'Latin America & Caribbean (WB)', 'South America', 'Sub-Saharan Africa (WB)', 'Central Asia', 'Eastern Mediterranean Region (WHO)', 'OECD Countries', 'East Asia & Pacific (WB)', 'North America', 'World', 'Europe & Central Asia (WB)', 'Middle East & North Africa (WB)', 'South Asia', 'Western Europe', 'Kosovo', 'Macau', 'Guadeloupe', 'Zaire', 'French Guiana', 'Hong Kong', 'Martinique', 'Bosnia-Herzegovina', 'New Caledonia', 'Serbia-Montenegro', 'East Timor', 'Wallis and Futuna', 'French Polynesia']

causes = causes[~causes['Country'].isin(causes_country_remove)]

len(set(causes['Country']))

#### Clean death age groups dataset

In [ ]:
age_groups = pd.read_csv('dataset_deaths_by_age_group.csv')

age_groups.head()

In [ ]:
# Clean column names

age_groups.columns = [re.sub('.+e: | \(.+', '', col) for col in age_groups.columns]

age_groups = age_groups.rename(columns = {'Entity': 'Country'})

age_groups.head()

In [ ]:
# Clean country-list

# print(set(age_groups['Country']))

In [ ]:
age_groups_country_remove = ['South Asia (WB)', 'African Region (WHO)', 'World Bank Upper Middle Income', 'World Bank Low Income', 'Western Pacific Region (WHO)', 'European Region (WHO)', 'World Bank High Income', 'England', 'Northern Ireland', 'Scotland', 'Wales', 'North America (WB)', 'World Bank Lower Middle Income', 'Region of the Americas (WHO)', 'G20', 'South-East Asia Region (WHO)', 'Latin America & Caribbean (WB)', 'Sub-Saharan Africa (WB)', 'Eastern Mediterranean Region (WHO)', 'OECD Countries', 'East Asia & Pacific (WB)', 'World', 'Europe & Central Asia (WB)', 'Middle East & North Africa (WB)']

age_groups = age_groups[~age_groups['Country'].isin(age_groups_country_remove)]

len(set(age_groups['Country']))

#### Merge datasets death causes + age groups

In [ ]:
causes_age_groups = pd.merge(causes, age_groups, left_on = ['Country', 'Code', 'Year'], right_on = ['Country', 'Code', 'Year'])

causes_age_groups.head()

#### Clean death categories dataset

In [ ]:
categories = pd.read_csv('dataset_deaths_by_category.csv')

categories.head()

In [ ]:
categories.columns

In [ ]:
# Clean column names

categories.columns = [re.sub('^.{8} | - S.+', '', col) for col in categories.columns]

categories = categories.rename(columns = {'Entity': 'Country'})

categories.head()

In [ ]:
# Clean country-list

# print(set(categories['Country']))

In [ ]:
categories_country_remove = ['World Bank Lower Middle Income', 'OECD Countries', 'G20', 'Scotland', 'England', 'Wales', 'World Bank Low Income', 'Western Pacific Region (WHO)', 'Region of the Americas (WHO)', 'South-East Asia Region (WHO)', 'World Bank High Income', 'North America (WB)', 'African Region (WHO)', 'Eastern Mediterranean Region (WHO)', 'World Bank Upper Middle Income', 'Latin America & Caribbean (WB)', 'Middle East & North Africa (WB)', 'Sub-Saharan Africa (WB)', 'Europe & Central Asia (WB)', 'European Region (WHO)', 'Northern Ireland', 'World', 'South Asia (WB)', 'East Asia & Pacific (WB)']

categories = categories[~categories['Country'].isin(categories_country_remove)]

len(set(categories['Country']))

#### Merge dataframe death causes/age-groups + categories

In [ ]:
deaths_df = pd.merge(causes_age_groups, categories, left_on = ['Country', 'Code', 'Year'], right_on = ['Country', 'Code', 'Year'])

deaths_df.head()

#### Merge dataframe death causes/age-groups/categories + gdp + population

In [ ]:
gdp_population = pd.read_excel('dataset_gdp_population.xlsx')

gdp_population.head()

In [ ]:
deaths_df = pd.merge(deaths_df, gdp_population, left_on = 'Country', right_on = 'Country')

deaths_df.head()

#### Calculate income classification

In [ ]:
# Low income            < 1.085
# Lower-middle income   1.086 - 4.255
# Upper-middle income   4.256 - 13.205
# High income           > 13.205

In [ ]:
def income_classification(income):
   if income < 1085:
      return 'Low'
   elif income < 4255:
      return 'Lower-middle'
   elif income < 13205:
      return 'Upper-middle'
   else:
      return 'High'

In [ ]:
deaths_df['Income'] = deaths_df['GDP_capita_USD'].apply(income_classification)

deaths_df.head()

#### Calculate crude death rate

In [ ]:
# Sum of deaths / Population * 100000

deaths_df['Death_rate_100K'] = (deaths_df.iloc[:, 36:41].sum(axis = 1) / deaths_df['Population'])

deaths_df.head()

In [ ]:
deaths_df.shape

In [ ]:
# Create Tableau dashboard view

deaths_df.to_excel('tableau_view.xlsx')